In [1]:
import pandas as pd
from tqdm import tqdm
from collections import Counter
from tqdm import tqdm
import random
from scipy import sparse
import numpy as np
import pandas as pd
from sklearn import preprocessing
import time
import tensorflow as tf

In [2]:
train_pd = pd.read_csv("./data/train.csv", encoding="utf-8")

In [3]:
word_list = []
vocab_list = []
for line in tqdm(train_pd.value):
    word_list.append([w for w in line])
    vocab_list.extend([w for w in line])

100%|█████████████████████████████████████████████████████████████████████████| 30000/30000 [00:00<00:00, 63390.18it/s]


In [4]:
c = Counter(vocab_list)
vocab = np.array(list(c.keys()))
vocab.sort()
vocab

array(['%', '&', '*', '+', '-', '.', '0', '1', '2', '3', '4', '5', '6',
       '7', '8', '9', '=', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I',
       'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V',
       'W', 'X', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h',
       'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
       'v', 'w', 'x', 'y', 'z'],
      dtype='<U1')

In [5]:
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(vocab)
label_encoder.classes_
label_encoder.transform(vocab)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69], dtype=int64)

In [6]:
word_labelencoder = [list(label_encoder.transform(w)) for w in word_list]
word_size = [len(i) for i in word_labelencoder]

In [7]:
train_size_pd = pd.DataFrame(word_size, columns=['len'])
train_size_pd['label'] = train_pd.label
train_size_pd.describe()

,len,label
count,30000.000000,30000.000000
mean,113.928533,0.083333
std,173.427420,0.276390
min,16.000000,0.000000
25%,53.000000,0.000000
50%,73.000000,0.000000
75%,100.000000,0.000000
max,11553.000000,1.000000


In [8]:
train_size_pd = train_size_pd[train_size_pd.len < 500]
Counter(train_size_pd.label)

Counter({0: 27159, 1: 1973})

In [9]:
pos_index = train_size_pd[train_size_pd.label == 1].index.values
random.shuffle(pos_index)
pos_index, pos_index.shape

(array([15906,  6747, 13894, ..., 26231, 12144, 16615], dtype=int64), (1973,))

In [10]:
neg_index = train_size_pd[train_size_pd.label == 0].index.values
neg_index = np.random.choice(neg_index, size=4000-1973)
neg_index, neg_index.shape

(array([23717, 25048, 16107, ..., 15726,  3560, 26484], dtype=int64), (2027,))

In [11]:
balance_train_index = np.append(pos_index,neg_index)
random.shuffle(balance_train_index)
balance_train_index, balance_train_index.shape

(array([ 5698, 23919,   825, ..., 11661, 16415, 14968], dtype=int64), (4000,))

In [12]:
x_batch = np.array(word_labelencoder.copy())
x_batch = x_batch[balance_train_index]
y_batch = np.array(train_pd.label.copy())
y_batch = y_batch[balance_train_index]
x_batch_size = [len(i) for i in x_batch]
x_batch.shape, y_batch.shape, len(x_batch_size)

((4000,), (4000,), 4000)

In [13]:
x_batch[:1]

array([ list([54, 48, 68, 58, 57, 48, 16, 49, 11, 7, 48, 12, 11, 7, 48, 12, 9, 45, 6, 14, 1, 54, 48, 68, 63, 66, 58, 16, 4, 7, 0, 8, 22, 2, 2, 0, 8, 22, 64, 57, 52, 58, 57, 0, 8, 22, 2, 2, 0, 8, 22, 62, 48, 55, 48, 46, 63, 0, 8, 22, 2, 2, 0, 8, 22, 6, 0, 8, 19, 7, 0, 8, 19, 8, 0, 8, 19, 9, 0, 8, 19, 10, 0, 8, 19, 11, 0, 8, 19, 46, 58, 57, 46, 44, 63, 0, 8, 14, 64, 52, 47, 0, 8, 19, 46, 51, 44, 61, 0, 8, 14, 8, 6, 0, 8, 15, 0, 8, 19, 64, 62, 48, 61, 57, 44, 56, 48, 0, 8, 19, 46, 51, 44, 61, 0, 8, 14, 8, 6, 0, 8, 15, 0, 8, 19, 59, 44, 62, 62, 66, 58, 61, 47, 0, 8, 15, 0, 8, 19, 13, 0, 8, 19, 14, 0, 8, 19, 65, 48, 61, 62, 52, 58, 57, 0, 8, 14, 0, 8, 15, 0, 8, 19, 7, 6, 0, 8, 19, 64, 62, 48, 61, 0, 8, 14, 0, 8, 15, 0, 8, 19, 7, 8, 0, 8, 19, 7, 9, 0, 8, 19, 7, 10, 0, 8, 19, 7, 11, 0, 8, 19, 7, 12, 0, 8, 19, 7, 13, 3, 49, 61, 58, 56, 3, 64, 62, 48, 61, 62, 0, 8, 22, 2, 1, 54, 48, 68, 63, 51, 61, 48, 48, 16, 13, 15, 14, 10, 8, 14, 12, 12, 12, 15])], dtype=object)

In [14]:
y_batch[:100]

array([1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 1], dtype=int64)

In [15]:
max_size = np.max(x_batch_size)
max_size

499

In [16]:
x_batch_pad = []
for x in tqdm(x_batch[:]):
    list_test = list()
    list_test =([-1] * max_size)
    list_test[:len(x)] = x
#     list_test = [[i] for i in list_test]
    x_batch_pad.append(list_test)

100%|███████████████████████████████████████████████████████████████████████████| 4000/4000 [00:00<00:00, 97300.96it/s]


In [17]:
x_batch_pad[:10]

[[54,
  48,
  68,
  58,
  57,
  48,
  16,
  49,
  11,
  7,
  48,
  12,
  11,
  7,
  48,
  12,
  9,
  45,
  6,
  14,
  1,
  54,
  48,
  68,
  63,
  66,
  58,
  16,
  4,
  7,
  0,
  8,
  22,
  2,
  2,
  0,
  8,
  22,
  64,
  57,
  52,
  58,
  57,
  0,
  8,
  22,
  2,
  2,
  0,
  8,
  22,
  62,
  48,
  55,
  48,
  46,
  63,
  0,
  8,
  22,
  2,
  2,
  0,
  8,
  22,
  6,
  0,
  8,
  19,
  7,
  0,
  8,
  19,
  8,
  0,
  8,
  19,
  9,
  0,
  8,
  19,
  10,
  0,
  8,
  19,
  11,
  0,
  8,
  19,
  46,
  58,
  57,
  46,
  44,
  63,
  0,
  8,
  14,
  64,
  52,
  47,
  0,
  8,
  19,
  46,
  51,
  44,
  61,
  0,
  8,
  14,
  8,
  6,
  0,
  8,
  15,
  0,
  8,
  19,
  64,
  62,
  48,
  61,
  57,
  44,
  56,
  48,
  0,
  8,
  19,
  46,
  51,
  44,
  61,
  0,
  8,
  14,
  8,
  6,
  0,
  8,
  15,
  0,
  8,
  19,
  59,
  44,
  62,
  62,
  66,
  58,
  61,
  47,
  0,
  8,
  15,
  0,
  8,
  19,
  13,
  0,
  8,
  19,
  14,
  0,
  8,
  19,
  65,
  48,
  61,
  62,
  52,
  58,
  57,
  0,
  8,
  14,
  0,
  8,
 

In [18]:
x_batch_pad = [np.eye(len(vocab))[item] for item in x_batch_pad]
# x_batch_pad = [list(i) for i in x_batch_pad]

In [19]:
x_batch_pad[0].shape

(499, 70)

In [20]:
x_batch_pad[:1]

[array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  1.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.]])]

In [21]:
y_batch_pad = [np.eye(2)[item] for item in y_batch]
y_batch_pad = [list(i) for i in y_batch_pad]

In [22]:
y_batch_pad

[[0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],

In [23]:
x_batch_size

[263,
 172,
 425,
 346,
 153,
 322,
 92,
 433,
 266,
 88,
 217,
 278,
 80,
 267,
 49,
 60,
 64,
 292,
 35,
 38,
 35,
 64,
 26,
 40,
 72,
 40,
 254,
 360,
 102,
 101,
 306,
 228,
 281,
 61,
 320,
 195,
 493,
 35,
 141,
 46,
 202,
 90,
 82,
 155,
 499,
 227,
 43,
 64,
 440,
 187,
 362,
 81,
 150,
 77,
 216,
 47,
 334,
 74,
 269,
 228,
 159,
 62,
 65,
 48,
 107,
 135,
 55,
 63,
 102,
 377,
 57,
 42,
 206,
 287,
 102,
 212,
 142,
 156,
 227,
 322,
 231,
 87,
 196,
 45,
 135,
 53,
 294,
 65,
 64,
 179,
 124,
 397,
 413,
 87,
 344,
 47,
 229,
 206,
 311,
 466,
 47,
 112,
 452,
 339,
 88,
 85,
 218,
 110,
 90,
 149,
 35,
 128,
 87,
 231,
 387,
 34,
 59,
 153,
 45,
 463,
 96,
 88,
 114,
 174,
 88,
 97,
 174,
 82,
 373,
 288,
 43,
 27,
 190,
 49,
 407,
 78,
 209,
 69,
 288,
 70,
 85,
 345,
 241,
 323,
 35,
 306,
 112,
 165,
 157,
 123,
 46,
 496,
 71,
 374,
 95,
 405,
 162,
 78,
 193,
 37,
 87,
 213,
 33,
 224,
 317,
 96,
 185,
 167,
 257,
 394,
 80,
 291,
 63,
 48,
 174,
 77,
 175,
 201,
 392,

In [24]:
class Batcher():
    def __init__(self, x, y, x_batch_size):
        self.split_size = int(len(x)*0.8)
        self.train_x = x[:self.split_size]
        self.train_y = y[:self.split_size]
        self.train_size = x_batch_size[:self.split_size]
        self.test_x = x[self.split_size:]
        self.test_y = y[self.split_size:]
        self.test_size = x_batch_size[self.split_size:]
        self.start = 0
    def next_batch(self, batch_size):
        s_index = self.start
        e_index = self.start + batch_size
        if e_index >= self.split_size:
            self.start = 0
            s_index = self.start
            e_index = self.start + batch_size
        self.start = e_index
        return self.train_x[s_index:e_index], self.train_y[s_index:e_index], self.train_size[s_index:e_index]

In [25]:
sqli_batch = Batcher(x_batch_pad, y_batch_pad, x_batch_size)

In [26]:
batch_data, batch_labels, batch_seqlen = sqli_batch.next_batch(10)

In [27]:
batch_data

[array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  1.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.]]),
 array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  1.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.]]),
 array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  1.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.]]),
 array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       

In [28]:
batch_labels

[[0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0]]

In [29]:
batch_seqlen

[263, 172, 425, 346, 153, 322, 92, 433, 266, 88]

In [32]:
tf.reset_default_graph()

# Parameters
learning_rate = 0.01
training_steps = 100
batch_size = 100
display_step = 10

# Network Parameters
seq_max_len = max_size # Sequence max length
n_hidden = 128 # hidden layer num of features
n_classes = 2 # linear sequence or not

# trainset = ToySequenceData(n_samples=1000, max_seq_len=seq_max_len)
# testset = ToySequenceData(n_samples=500, max_seq_len=seq_max_len)
sqli_batch = Batcher(x_batch_pad, y_batch_pad, x_batch_size)

# tf Graph input
x = tf.placeholder("float", [None, seq_max_len, len(vocab)])
y = tf.placeholder("float", [None, n_classes])
# A placeholder for indicating each sequence length
seqlen = tf.placeholder(tf.int32, [None])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}


def dynamicRNN(x, seqlen, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, seq_max_len, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)

    # Get lstm cell output, providing 'sequence_length' will perform dynamic
    # calculation.
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32,
                                sequence_length=seqlen)

    # When performing dynamic calculation, we must retrieve the last
    # dynamically computed output, i.e., if a sequence length is 10, we need
    # to retrieve the 10th output.
    # However TensorFlow doesn't support advanced indexing yet, so we build
    # a custom op that for each sample in batch size, get its length and
    # get the corresponding relevant output.

    # 'outputs' is a list of output at every timestep, we pack them in a Tensor
    # and change back dimension to [batch_size, n_step, n_input]
    outputs = tf.stack(outputs)
    outputs = tf.transpose(outputs, [1, 0, 2])

    # Hack to build the indexing and retrieve the right output.
    batch_size = tf.shape(outputs)[0]
    # Start indices for each sample
    index = tf.range(0, batch_size) * seq_max_len + (seqlen - 1)
    # Indexing
    outputs = tf.gather(tf.reshape(outputs, [-1, n_hidden]), index)

    # Linear activation, using outputs computed above
    return tf.matmul(outputs, weights['out']) + biases['out']

print("before drnn")
pred = dynamicRNN(x, seqlen, weights, biases)
print("after drnn")

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

before drnn
after drnn


c:\program files\python36\lib\site-packages\tensorflow\python\ops\gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [39]:
LOG_DIR = "./dlstm/"
saver = tf.train.Saver(tf.global_variables(), max_to_keep=15)
module_file = tf.train.latest_checkpoint(LOG_DIR)

# Start training
with tf.Session() as sess:

    # Run the initializer
#     sess.run(init)
    saver.restore(sess, module_file)

    for step in range(1, training_steps + 1):
        # Get batch data
        batch_x, batch_y, batch_seqlen = sqli_batch.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       seqlen: batch_seqlen})
        if step % display_step == 0 or step == 1:
            # Calculate batch accuracy & loss
            acc, loss = sess.run([accuracy, cost], feed_dict={x: batch_x, y: batch_y,
                                                seqlen: batch_seqlen})
            print("Step " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
            # Calculate accuracy
            test_data = sqli_batch.test_x
            test_label = sqli_batch.test_y
            test_seqlen = sqli_batch.test_size
            test_acc = sess.run(accuracy, feed_dict={x: test_data, y: test_label, seqlen: test_seqlen})
            print("Testing Accuracy:", test_acc)
    
        if step % 1 == 0:
            saver.save(sess, LOG_DIR+"sqli.ckpt", global_step=step)

    print("Optimization Finished!")

    # Calculate accuracy
    test_data = sqli_batch.test_x
    test_label = sqli_batch.test_y
    test_seqlen = sqli_batch.test_size
    test_acc = sess.run(accuracy, feed_dict={x: test_data, y: test_label, seqlen: test_seqlen})
    print("Testing Accuracy:", test_acc)

INFO:tensorflow:Restoring parameters from ./dlstm/sqli.ckpt-100
Step 100, Minibatch Loss= 0.144621, Training Accuracy= 0.97000
Testing Accuracy: 0.955
Step 1000, Minibatch Loss= 1.044556, Training Accuracy= 0.55000
Testing Accuracy: 0.50125
Step 2000, Minibatch Loss= 0.112224, Training Accuracy= 0.99000
Testing Accuracy: 0.94375
Step 3000, Minibatch Loss= 0.219028, Training Accuracy= 0.93000
Testing Accuracy: 0.9475
Step 4000, Minibatch Loss= 0.146265, Training Accuracy= 0.96000
Testing Accuracy: 0.95
Step 5000, Minibatch Loss= 0.216743, Training Accuracy= 0.94000
Testing Accuracy: 0.9575
Step 6000, Minibatch Loss= 0.048493, Training Accuracy= 0.99000
Testing Accuracy: 0.95375
Step 7000, Minibatch Loss= 0.158803, Training Accuracy= 0.95000
Testing Accuracy: 0.95625
Step 8000, Minibatch Loss= 0.269696, Training Accuracy= 0.95000
Testing Accuracy: 0.9375
Step 9000, Minibatch Loss= 0.136029, Training Accuracy= 0.96000
Testing Accuracy: 0.955
Step 10000, Minibatch Loss= 0.122467, Training 

In [ ]:
INFO:tensorflow:Restoring parameters from ./dlstm/sqli.ckpt-100
Step 100, Minibatch Loss= 0.236982, Training Accuracy= 0.93000
Testing Accuracy: 0.9525
Step 1000, Minibatch Loss= 0.099061, Training Accuracy= 0.98000
Testing Accuracy: 0.9525
Step 2000, Minibatch Loss= 0.070561, Training Accuracy= 0.98000
Testing Accuracy: 0.955
Step 3000, Minibatch Loss= 0.195829, Training Accuracy= 0.95000
Testing Accuracy: 0.94375
Step 4000, Minibatch Loss= 0.112261, Training Accuracy= 0.98000
Testing Accuracy: 0.95375
Step 5000, Minibatch Loss= 0.194462, Training Accuracy= 0.95000
Testing Accuracy: 0.955
Step 6000, Minibatch Loss= 0.091309, Training Accuracy= 0.98000
Testing Accuracy: 0.955
Step 7000, Minibatch Loss= 0.147598, Training Accuracy= 0.96000
Testing Accuracy: 0.95625
Step 8000, Minibatch Loss= 0.128473, Training Accuracy= 0.95000
Testing Accuracy: 0.95625
Step 9000, Minibatch Loss= 0.133661, Training Accuracy= 0.96000
Testing Accuracy: 0.95625
Step 10000, Minibatch Loss= 0.087887, Training Accuracy= 0.98000
Testing Accuracy: 0.955
Optimization Finished!
Testing Accuracy: 0.955

In [ ]:
INFO:tensorflow:Restoring parameters from ./dlstm/sqli.ckpt-100
Step 100, Minibatch Loss= 0.126827, Training Accuracy= 0.98000
Testing Accuracy: 0.9475
Step 1000, Minibatch Loss= 0.128920, Training Accuracy= 0.96000
Testing Accuracy: 0.95125
Step 2000, Minibatch Loss= 0.151861, Training Accuracy= 0.96000
Testing Accuracy: 0.95375
Step 3000, Minibatch Loss= 0.151962, Training Accuracy= 0.96000
Testing Accuracy: 0.94875
Step 4000, Minibatch Loss= 0.099038, Training Accuracy= 0.98000
Testing Accuracy: 0.9575
Step 5000, Minibatch Loss= 0.148384, Training Accuracy= 0.96000
Testing Accuracy: 0.9525
Step 6000, Minibatch Loss= 0.218183, Training Accuracy= 0.95000
Testing Accuracy: 0.95375
Step 7000, Minibatch Loss= 0.264490, Training Accuracy= 0.92000
Testing Accuracy: 0.95625
Step 8000, Minibatch Loss= 0.194151, Training Accuracy= 0.95000
Testing Accuracy: 0.95375
Step 9000, Minibatch Loss= 0.270325, Training Accuracy= 0.93000
Testing Accuracy: 0.90875
Step 10000, Minibatch Loss= 0.126782, Training Accuracy= 0.96000
Testing Accuracy: 0.95375
Optimization Finished!
Testing Accuracy: 0.95375

In [ ]:
INFO:tensorflow:Restoring parameters from ./dlstm/sqli.ckpt-100
Step 100, Minibatch Loss= 0.165025, Training Accuracy= 0.95000
Testing Accuracy: 0.95125
Step 1000, Minibatch Loss= 0.097430, Training Accuracy= 0.97000
Testing Accuracy: 0.9525
Step 2000, Minibatch Loss= 0.226068, Training Accuracy= 0.93000
Testing Accuracy: 0.95125
Step 3000, Minibatch Loss= 0.153346, Training Accuracy= 0.96000
Testing Accuracy: 0.9475
Step 4000, Minibatch Loss= 0.290856, Training Accuracy= 0.90000
Testing Accuracy: 0.945
Step 5000, Minibatch Loss= 0.056507, Training Accuracy= 0.99000
Testing Accuracy: 0.95
Step 6000, Minibatch Loss= 0.157843, Training Accuracy= 0.95000
Testing Accuracy: 0.95125
Step 7000, Minibatch Loss= 0.120065, Training Accuracy= 0.97000
Testing Accuracy: 0.94625
Step 8000, Minibatch Loss= 0.150518, Training Accuracy= 0.96000
Testing Accuracy: 0.9525
Step 9000, Minibatch Loss= 0.131259, Training Accuracy= 0.97000
Testing Accuracy: 0.9525
Step 10000, Minibatch Loss= 0.038164, Training Accuracy= 1.00000
Testing Accuracy: 0.95125
Optimization Finished!
Testing Accuracy: 0.95125

In [ ]:
INFO:tensorflow:Restoring parameters from ./dlstm/sqli.ckpt-100
Step 100, Minibatch Loss= 0.214280, Training Accuracy= 0.94000
Testing Accuracy: 0.9475
Step 1000, Minibatch Loss= 0.107233, Training Accuracy= 0.98000
Testing Accuracy: 0.9525
Step 2000, Minibatch Loss= 0.197336, Training Accuracy= 0.94000
Testing Accuracy: 0.945
Step 3000, Minibatch Loss= 0.130594, Training Accuracy= 0.97000
Testing Accuracy: 0.93875
Step 4000, Minibatch Loss= 0.212512, Training Accuracy= 0.94000
Testing Accuracy: 0.95
Step 5000, Minibatch Loss= 0.100070, Training Accuracy= 0.98000
Testing Accuracy: 0.95375
Step 6000, Minibatch Loss= 0.161161, Training Accuracy= 0.96000
Testing Accuracy: 0.9525
Step 7000, Minibatch Loss= 0.154573, Training Accuracy= 0.95000
Testing Accuracy: 0.9525
Step 8000, Minibatch Loss= 0.144155, Training Accuracy= 0.96000
Testing Accuracy: 0.95375
Step 9000, Minibatch Loss= 0.091603, Training Accuracy= 0.97000
Testing Accuracy: 0.9525
Step 10000, Minibatch Loss= 0.152378, Training Accuracy= 0.96000
Testing Accuracy: 0.94875
Optimization Finished!
Testing Accuracy: 0.94875